In [1]:
import pandas as pd

ground_truth_df = pd.read_json('evaluation_ground_truth.json')
ground_truth_df

,ground_truth_faq_id,generated_question,ground_truth_question,ground_truth_answer,ground_truth_courier
0,0,What type of employment contract do I have wit...,What is my contract type as an employee?,"As an employee, you will have a part-time or f...","{'index': 2, 'first_name': 'Olivia', 'last_nam..."
1,0,What benefits do I receive as an employee?,What is my contract type as an employee?,"As an employee, you will have a part-time or f...","{'index': 2, 'first_name': 'Olivia', 'last_nam..."
2,0,Am I eligible for paid vacation as an employee?,What is my contract type as an employee?,"As an employee, you will have a part-time or f...","{'index': 2, 'first_name': 'Olivia', 'last_nam..."
3,0,What distinguishes my contract from that of a ...,What is my contract type as an employee?,"As an employee, you will have a part-time or f...","{'index': 2, 'first_name': 'Olivia', 'last_nam..."
4,0,Does being an employee affect my salary struct...,What is my contract type as an employee?,"As an employee, you will have a part-time or f...","{'index': 2, 'first_name': 'Olivia', 'last_nam..."
...,...,...,...,...,...
1920,654,Am I allowed to decline deliveries to public b...,Can I refuse to deliver to a public building?,"No, you must deliver to the specified address....","{'index': 0, 'first_name': 'Emma', 'last_name'..."
1921,654,What is my age as per my courier profile?,Can I refuse to deliver to a public building?,"No, you must deliver to the specified address....","{'index': 0, 'first_name': 'Emma', 'last_name'..."
1922,654,What type of contract do I have with iDelivery?,Can I refuse to deliver to a public building?,"No, you must deliver to the specified address....","{'index': 0, 'first_name': 'Emma', 'last_name'..."
1923,654,Can I choose my own vehicle type for deliveries?,Can I refuse to deliver to a public building?,"No, you must deliver to the specified address....","{'index': 0, 'first_name': 'Emma', 'last_name'..."


In [37]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def vector_search(question, country, score_threshold, limit):
    # print('vector_search is called on question: '+question)
    
    return qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="country",
                    match=models.MatchAny(any=[country, "all"] )
                )
            ]
        ),
        score_threshold = score_threshold,
        limit=limit,
        with_payload=True
    )


In [12]:
from qdrant_client import QdrantClient, models

qd_client = QdrantClient("http://localhost:6333")
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "courier-faq"


In [39]:
from tqdm.auto import tqdm

def evaluate(ground_truth_df, search_function):
    relevance_total = []

    for ground_truth in tqdm(ground_truth_df.to_dict('records')):
        doc_id = ground_truth['ground_truth_faq_id']

        generated_question = ground_truth['generated_question']
        country = ground_truth['ground_truth_courier']['country']
        
        result_list = search_function(generated_question, country)
        relevance = [result.id == doc_id for result in result_list.points]
        relevance_total.append(relevance)

        # print("ground_truth_faq_id: ",doc_id)
        # print("generated_question: ",generated_question)
        # print("ground_truth_question: ",ground_truth['ground_truth_question'])
        # print("country: ",country)
        # print("result_list.points: ",result_list.points)
        # print("relevance: ",relevance)
        # print()
        
        # break

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [59]:

evaluate(sampled_ground_truth_df, lambda question, country: vector_search(question, country, 0.8, 5))

  0%|          | 0/100 [00:00<?, ?it/s]

{'hit_rate': 0.88, 'mrr': 0.8366666666666666}

In [75]:
import numpy as np

score = []
sampled_ground_truth_df = ground_truth_df.head(1000)

for score_threshold in np.arange(0.4, 1.0, 0.1):
    for limit in np.arange(5, 10, 1):
        print("Evaluating cofigs: score_threshold="+str(score_threshold)+", limit="+str(limit))
        result = evaluate(sampled_ground_truth_df, lambda question, country: vector_search(question, country, score_threshold, limit))
        print("Result: ", result)
        score.append({
            "hit_rate": result['hit_rate'],
            "mrr": result['mrr'],
            "score_threshold": score_threshold,
            "limit": limit,
        })

result = sorted(score, key=lambda item: item['mrr'], reverse=True)
print("Best params: ")
result


Evaluating cofigs: score_threshold=0.4, limit=5


  0%|          | 0/1000 [00:00<?, ?it/s]

Result:  {'hit_rate': 0.718, 'mrr': 0.5571166666666686}
Evaluating cofigs: score_threshold=0.4, limit=6


  0%|          | 0/1000 [00:00<?, ?it/s]

Result:  {'hit_rate': 0.743, 'mrr': 0.5621833333333348}
Evaluating cofigs: score_threshold=0.4, limit=7


...

  0%|          | 0/1000 [00:00<?, ?it/s]

Result:  {'hit_rate': 0.178, 'mrr': 0.13670833333333327}
Evaluating cofigs: score_threshold=0.8999999999999999, limit=9


  0%|          | 0/1000 [00:00<?, ?it/s]

Result:  {'hit_rate': 0.18, 'mrr': 0.13709563492063484}
Best params: 


[{'hit_rate': 0.83,
  'mrr': 0.5727579365079365,
  'score_threshold': np.float64(0.4),
  'limit': np.int64(9)},
 {'hit_rate': 0.83,
  'mrr': 0.5727579365079365,
  'score_threshold': np.float64(0.5),
  'limit': np.int64(9)},
 {'hit_rate': 0.83,
  'mrr': 0.5727579365079365,
  'score_threshold': np.float64(0.6),
  'limit': np.int64(9)},
 {'hit_rate': 0.83,
  'mrr': 0.5727579365079365,
  'score_threshold': np.float64(0.7),
  'limit': np.int64(9)},
 {'hit_rate': 0.811,
  'mrr': 0.5666642857142864,
  'score_threshold': np.float64(0.4),
  'limit': np.int64(8)},
 {'hit_rate': 0.811,
  'mrr': 0.5666642857142864,
  'score_threshold': np.float64(0.5),
  'limit': np.int64(8)},
 {'hit_rate': 0.811,
  'mrr': 0.5666642857142864,
  'score_threshold': np.float64(0.6),
  'limit': np.int64(8)},
 {'hit_rate': 0.811,
  'mrr': 0.5666642857142864,
  'score_threshold': np.float64(0.7),
  'limit': np.int64(8)},
 {'hit_rate': 0.792,
  'mrr': 0.564792857142858,
  'score_threshold': np.float64(0.4),
  'limit': np